In [26]:
import SimpleITK as sitk
import numpy as np
import os

img_ref = sitk.ReadImage('/home/viscent/hdd/viscent/dHCP/synthsr_data/1-SR_real/t1_input_1.nii.gz')
img_t1 = sitk.ReadImage('/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T1_hf_brain.nii.gz')
img_t2 = sitk.ReadImage('/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T2_hf_brain.nii.gz')

In [27]:
import SimpleITK as sitk
import skimage
from scipy.ndimage import zoom
print(img_t1.GetSpacing())
print(img_t1.GetSize())
# sitk.Show(img_t1)
img_t1_arr = sitk.GetArrayFromImage(img_t1)
img_t1_arr = zoom(img_t1_arr, img_t1.GetSpacing()[::-1])
# img_t1_arr = skimage.transform.rescale(img_t1_arr,img_t1.GetSpacing())
# img_t1.SetSpacing((1.0, 1.0, 1.0))
# img_t2.SetSpacing((1.0, 1.0, 1.0))
# img_t1=resize_image_itk(img_t1, img_ref)
# sitk.WriteImage(img_t1, '/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T1_hf_brain_resized.nii.gz')
sitk.Show(img_t1)
# sitk.WriteImage(img_t2, '/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T2_hf_brain_resized.nii.gz')

(1.5099999904632568, 1.5, 5.0)
(120, 146, 36)


MetaImageIO (0x7f6ab38b4610): Unsupported or empty metaData item intent_name of type Ssfound, won't be written to image file

OpenJDK 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
OpenJDK 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


[INFO] Detected existing ImageJ; passing arguments along


In [12]:
def resize_image_itk(ori_img, target_img, resamplemethod=sitk.sitkLinear):
    """
    用itk方法将原始图像resample到与目标图像一致
    :param ori_img: 原始需要对齐的itk图像
    :param target_img: 要对齐的目标itk图像
    :param resamplemethod: itk插值方法: sitk.sitkLinear-线性  sitk.sitkNearestNeighbor-最近邻
    :return:img_res_itk: 重采样好的itk图像
    """
    target_Size = []
    for a,b in zip(target_img.GetSize(), target_img.GetSpacing()):
        target_Size.append(int(a*b))

    # target_Size = ori_img.GetSize()*ori_img.GetSpacing()    # 目标图像大小  [x,y,z]
    target_Spacing = target_img.GetSpacing()   # 目标的体素块尺寸    [x,y,z]
    target_origin = ori_img.GetOrigin()      # 目标的起点 [x,y,z]
    target_direction = ori_img.GetDirection()  # 目标的方向 [冠,矢,横]=[z,y,x]

    print('ori_size:', ori_img.GetSize())
    print('target_Size:', target_Size)

    # itk的方法进行resample
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ori_img)  # 需要重新采样的目标图像
    # 设置目标图像的信息
    resampler.SetSize(target_Size)		# 目标图像大小
    resampler.SetOutputOrigin(target_origin)
    resampler.SetOutputDirection(target_direction)
    resampler.SetOutputSpacing(target_Spacing)
    # 根据需要重采样图像的情况设置不同的dype
    if resamplemethod == sitk.sitkNearestNeighbor:
        resampler.SetOutputPixelType(sitk.sitkUInt16)   # 近邻插值用于mask的，保存uint16
    else:
        resampler.SetOutputPixelType(sitk.sitkFloat32)  # 线性插值用于PET/CT/MRI之类的，保存float32
    resampler.SetTransform(sitk.Transform(3, sitk.sitkAffine))    
    resampler.SetInterpolator(resamplemethod)
    itk_img_resampled = resampler.Execute(ori_img)  # 得到重新采样后的图像
    return itk_img_resampled

In [18]:
def resize_image_itk_ref(ori_img, target_img, resamplemethod=sitk.sitkLinear):
    """
    用itk方法将原始图像resample到与目标图像一致
    :param ori_img: 原始需要对齐的itk图像
    :param target_img: 要对齐的目标itk图像
    :param resamplemethod: itk插值方法: sitk.sitkLinear-线性  sitk.sitkNearestNeighbor-最近邻
    :return:img_res_itk: 重采样好的itk图像
    """
    target_Size = target_img.GetSize()      # 目标图像大小  [x,y,z]
    target_Spacing = target_img.GetSpacing()   # 目标的体素块尺寸    [x,y,z]
    target_origin = ori_img.GetOrigin()      # 目标的起点 [x,y,z]
    target_direction = target_img.GetDirection()  # 目标的方向 [冠,矢,横]=[z,y,x]

    print('ori_size:', ori_img.GetSize())
    print('target_Size:', target_Size)

    # itk的方法进行resample
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ori_img)  # 需要重新采样的目标图像
    # 设置目标图像的信息
    resampler.SetSize(target_Size)		# 目标图像大小
    resampler.SetOutputOrigin(target_origin)
    resampler.SetOutputDirection(target_direction)
    resampler.SetOutputSpacing(target_Spacing)
    # 根据需要重采样图像的情况设置不同的dype
    if resamplemethod == sitk.sitkNearestNeighbor:
        resampler.SetOutputPixelType(sitk.sitkUInt16)   # 近邻插值用于mask的，保存uint16
    else:
        resampler.SetOutputPixelType(sitk.sitkFloat32)  # 线性插值用于PET/CT/MRI之类的，保存float32
    resampler.SetTransform(sitk.Transform(3, sitk.sitkAffine))    
    resampler.SetInterpolator(resamplemethod)
    itk_img_resampled = resampler.Execute(ori_img)  # 得到重新采样后的图像
    return itk_img_resampled

In [19]:
import copy
img_t1_resized = resize_image_itk(img_t1, img_ref)
img_t2_resized = resize_image_itk(img_t2, img_ref)
aug_image = copy.copy(img_t1_resized)
aug_image = resize_image_itk_ref(aug_image, img_ref)

sitk.WriteImage(img_t1_resized, '/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T1_hf_brain_resized.nii.gz')
sitk.WriteImage(img_t2_resized, '/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T2_hf_brain_resized.nii.gz')
sitk.WriteImage(aug_image, '/home/viscent/hdd/viscent/SynthSR/data/CHILD/CHILD_T1_hf_brain_resized_aug.nii.gz')

ori_size: (120, 146, 36)
target_Size: [181, 219, 180]
ori_size: (120, 146, 36)
target_Size: [181, 219, 180]
ori_size: (181, 219, 180)
target_Size: (108, 145, 145)
